In [77]:
import requests
import csv
from decouple import config
key = config('KEY')
movie_list = []
with open('boxoffice.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        movie_list.append(row['영화 대표코드'])

In [92]:
movies = {}
for code in movie_list:
    api_url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key={key}&movieCd={code}'
    response = requests.get(api_url).json()
    movies[code] = {}
    movie = movies[code]
    movie_info = response['movieInfoResult']['movieInfo']
    movie_info_list = {'movieCd': '영화 대표코드', 'movieNm': '영화명(국문)', 'movieNmEn': '영화명(영문)','movieNmOg': '영화명(원문)','watchGradeNm': '관람등급','openDt': '개봉연도','showTm': '상영시간', 'genres': '장르','directors': '감독명'}
    for info, info_ko in movie_info_list.items():
        if info == 'watchGradeNm':
            if movie_info['audits'] != []:
                movie[info_ko] = movie_info['audits'][0][info]
            else:
                movie[info_ko] = ''
        elif info == 'genres':
            genrenm = []
            for genre in range(len(movie_info[info])):
                genrenm.append(movie_info[info][genre]['genreNm'])
            movie[info_ko] = ', '.join(genrenm)
        elif info == 'directors':
            directors = []
            for director in range(len(movie_info[info])):
                directors.append(movie_info[info][director]['peopleNm'])
            movie[info_ko] = ', '.join(directors)
        else:      
            movie[info_ko] = movie_info[info]

In [93]:
with open ('movie.csv', 'w', encoding='utf-8') as f:
    fieldnames = ['영화 대표코드','영화명(국문)', '영화명(영문)', '영화명(원문)', '관람등급', '개봉연도', '상영시간', '장르', '감독명']
    csv_writer = csv.DictWriter(f, fieldnames=fieldnames)
    csv_writer.writeheader()
    for item in movies.values(): # 딕셔너리 만든 것 반복
        csv_writer.writerow(item)